In [1]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from helpers.interpolate_extinction_coeffs import extinction_coeffs
from helpers.mci_files import create_layer_dict, createMciString

n = 1.4
g = 0.9
csf_scatter = 1.0
csf_abs = 0.0
water_moles_per_liter = 55.5
wls = np.arange(740, 902, 2) 
_, (water_ex, oxy_ex, deoxy_ex, lipid_ex) = extinction_coeffs(wls)

N = 500

# maybe need to include melanin afterall
with pm.Model() as variable_layers:
    blood = pm.Uniform('blood', lower=1, upper=100) # uM
    # blood O2 saturation
    blood_sat = pm.Beta('blood_sat', alpha=5, beta=2) # %
    oxy = pm.Deterministic('oxy', blood_sat*blood) # uM
    deoxy = pm.Deterministic('deoxy', (1 - blood_sat)*blood) #u uM

    # fractions as percentages
    lw_frac = pm.Beta('LW_frac', alpha=5, beta=2)
    w_over_lw_frac = pm.Beta('water_over_lipid_frac', alpha=2, beta=2)
    water_frac = pm.Deterministic('water_frac', lw_frac*w_over_lw_frac)
    lipid_frac = pm.Deterministic('lipid_frac', w_over_lw_frac - water_frac)

    # abs all in cm-1
    water_abs = pm.Deterministic('water_abs', water_frac*water_moles_per_liter*water_ex) 
    oxy_abs = pm.Deterministic('oxy_abs', oxy*oxy_ex*1e-6)
    deoxy_abs = pm.Deterministic('deoxy_abs', deoxy*deoxy_ex*1e-6)
    lipid_abs = pm.Deterministic('lipid_abs', lipid_frac*lipid_ex)
    absorption = pm.Deterministic('absorption',
        water_abs + oxy_abs + deoxy_abs + lipid_abs
    )

    a = pm.Uniform('a', lower=5, upper=60)
    b = pm.Uniform('b', lower=0.2, upper=3)

    scattering = pm.Deterministic('scattering',
        a*(wls/500.0)**(-b)/(1-g)
    )

    samples = pm.sample_prior_predictive(samples=N*2)['prior']

Sampling: [LW_frac, a, b, blood, blood_sat, water_over_lipid_frac]


Importantly, the following code cell is used to either set constant csf and skin-scalp layer sizes (for analysis in abc_fixed) or sample these heights from distributions (for analysis in abc.ipynb)

The irrelevant lines should be commented out

In [ ]:
sampleCSFHeight = lambda: pm.draw(pm.Uniform.dist(lower=0.1, upper=1.0))
sampleSkinScalpHeight = lambda: pm.draw(pm.Uniform.dist(lower=0.5, upper=1.5))

#sampleCSFHeight = lambda: 0.2
#sampleSkinScalpHeight = lambda: 1.0

In [6]:
import pandas as pd
import string
import random

add_postfix = lambda arr, postfix: [e + postfix for e in arr]
extract_keys = lambda sample, keys: [float(sample[e]) for e in keys]
random_id = lambda: "".join(random.choice(string.ascii_lowercase) for i in range(12))

bottom_height = 1e8

shared_props = "n", "g"
variable_layer_props = "blood", "blood_sat", "a", "b", "lipid_frac", "water_frac", "height"
constant_layer_props = "height", "mu_a", "mu_s"

file_postfix = "_hello_world"

col_names = [
    "uuid", *shared_props, *add_postfix(variable_layer_props, '_one'), *add_postfix(variable_layer_props, '_two'),
    *add_postfix(constant_layer_props, '_csf')
]

def createRun(samples):
    extract_layer = lambda sample, height: \
        [(n, float(sample['absorption'][i]), float(sample['scattering'][i]), g, height)
            for i in range(sample.dims['absorption_dim_0'])]

    df = pd.DataFrame(columns=col_names)

    draw_dim = samples.dims['draw']
    wls_dim = samples.dims['absorption_dim_0']
    for i in range(0, draw_dim, 2):
        file_id = random_id()

        skin_scalp_height = float(sampleSkinScalpHeight())
        csf_height = float(sampleCSFHeight())

        draw_one = samples.isel(chain=0, draw=i)
        draw_two = samples.isel(chain=0, draw=i+1)

        skin_scalp_layer = extract_layer(draw_one, skin_scalp_height)
        csf_layer = [(n, csf_abs, csf_scatter, g, csf_height)]*wls_dim
        brain_layer = extract_layer(draw_two, bottom_height)

        layers = [[create_layer_dict(*i), create_layer_dict(*j), create_layer_dict(*k)]
            for i, j, k in zip(skin_scalp_layer, csf_layer, brain_layer)]
        file_names = [f"output{file_postfix}/" + file_id + f"_{i}" for i in range(wls_dim)]

        part_one = extract_keys(draw_one, variable_layer_props[:-1]) + [skin_scalp_height]
        part_csf = [csf_height, csf_abs, csf_scatter]
        part_two = extract_keys(draw_two, variable_layer_props[:-1]) + [bottom_height]

        df.loc[len(df)] =[file_id, n, g, *part_one, *part_two, *part_csf]
        
        with open(f"./sims/mci{file_postfix}/{file_id}.mci", 'w') as f:
            f.write(createMciString(layers, file_names))
    return df

df = createRun(samples)

df.to_csv(f"./sims/df{file_postfix}.csv")